# Intro

{Fill in with information about this notebook}

# Set Up notebook 

In [1]:
#Import modules
import numpy as np #Data manipulation
import pandas as pd #Point data manipulation and organization
import xarray as xr #Raster data manipulation and organization

import pathlib  #For filepaths, io, etc.
import os       #For several system-based commands
import datetime #For manipulation of time data, including file creation/modification times
import json     #For dictionary io, etc.

import matplotlib.pyplot as plt #For plotting and data vizualization
import geopandas as gpd         #For organization and manipulation of vector data in space (study area and some data points)
import rioxarray as rxr         #For orgnaization and manipulation of raster data
import shapely                  #For converting coordinates to point geometry

#Scripts with functions made for this specific application
import w4h

#Variables needed throughout, best to just assign now
todayDate, dateSuffix = w4h.getCurrentDate() 
repoDir = pathlib.Path(os.getcwd())

# Read in data

- Set up filepaths
- Read in data from:
    - downholeData table (from database)
    - headerData table (from database)
    - xyzData file (from previously carried out work) (will eventually make this updateable)

Read in data

In [2]:
directoryDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\\'[:-1]
downholeDataPATH, headerDataPATH, xyzInPATH  = w4h.filesSetup(db_dir=directoryDir)

#Functions to read data into dataframes. Also excludes extraneous columns, and drops header data with no location information
headerDataIN, downholeDataIN = w4h.readRawTxtData(downholefile=downholeDataPATH, headerfile=headerDataPATH) 
xyzDataIN = w4h.readXYZData(xyzfile=xyzInPATH)

Most Recent version of this file is : ISGS_DOWNHOLE_DATA_2023-01-06.txt
Most Recent version of this file is : ISGS_HEADER_2023-01-06.txt
Most Recent version of this file is : xyzData.csv
Using the following files:

\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\ISGS_DOWNHOLE_DATA_2023-01-06.txt
\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\ISGS_HEADER_2023-01-06.txt
\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\xyzData.csv
Downhole Data has 3054409 valid well records.
Header Data has 636855 unique wells with valid location information.


Define datatypes (doing this during the read in process has presented issues)

In [3]:
#Define datatypes, to read into defineDataTypes() function
##EVENTUALLY, MAKE THIS A FILE IN THE RES FOLDER TO READ IN
#downholeDataDTYPES = {'ID':np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}
#headerDataDTYPES = {'ID':np.uint32,'API_NUMBER':np.uint64,"TDFORMATION":str,"PRODFORM":str,"TOTAL_DEPTH":np.float64,"SECTION":np.float64,"TWP":np.float64,"TDIR":str,"RNG":np.float64,"RDIR":str,"MERIDIAN":np.float64,"FARM_NAME":str,"NSFOOT":np.float64,"NSDIR":str,"EWFOOT":np.float64,"EWDIR":str,"QUARTERS":str,"ELEVATION":np.float64,"ELEVREF":str,"COMP_DATE":str,"STATUS":str,"FARM_NUM":str,"COUNTY_CODE":np.float64,"PERMIT_NUMBER":str,"COMPANY_NAME":str,"COMPANY_CODE":str,"PERMIT_DATE":str,"CORNER":str,"LATITUDE":np.float64,"LONGITUDE":np.float64,"ENTERED_BY":str,"UPDDATE":str,"ELEVSOURCE":str, "ELEV_FT":np.float64}
#xyzDataDTYPES = {'ID':np.uint64, 'API_NUMBER':np.uint64, "LATITUDE":np.float64, "LONGITUDE":np.float64, "ELEV_FT":np.float64}

#Define datatypes of each column of the new dataframes
downholeDataIN = w4h.defineDataTypes(downholeDataIN, dtypeFile='downholeDataTypes.txt')
headerDataIN = w4h.defineDataTypes(headerDataIN, dtypeFile='headerDataTypes.txt')
xyzDataIN = w4h.defineDataTypes(xyzDataIN, dtypeFile='xyzDataTypes.txt')

#Make a copy of the data so raw data is preserved while we work with the rest of the data
downholeData = downholeDataIN.copy()
headerData = headerDataIN.copy()
xyzData = xyzDataIN.copy()

Read in Study Area

In [1]:
import w4h
studyAreaPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ESL_StudyArea_5mi.shp"
#studyAreaPath = r"C:\Users\balikian\OneDrive - University of Illinois - Urbana\Data_OneDrive\CodesScripts\MahometBoreholesPolygon.zip"
studyAreaIN, saExtent = w4h.readStudyArea(studyAreaPath)

Read in/Define Model Grid, surface elevation grid, and bedrock elevation grid

In [2]:
import w4h

modelGridPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\grid_625_raster.tif"
surfaceElevPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ILStateLidar_ClipExtentESL.tif"
bedrockElevPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ESLBedrock.tif"

modelGrid = w4h.read_grid(datapath=modelGridPath, grid_type='model', studyArea=studyAreaIN,  read_grid=True, clip2SA=True)#, gridcrs='EPSG:26715', studyAreacrs='EPSG:26715')
surfaceElevGridIN = w4h.read_grid(datapath=surfaceElevPath, grid_type='surface', studyArea=studyAreaIN, use_service=False, clip2SA=True)
bedrockElevGridIN = w4h.read_grid(bedrockelevpath=bedrockElevPath, grid_type='bedrock', studyArea=studyAreaIN, use_service=False, clip2SA=True)
modelGrid

[2757394.3366691  1974361.24422738 2884613.47200259 2166018.56635791]
PROJCS["WGS 84 / Pseudo-Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_1SP"],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs"],AUTHORITY["EPSG","3857"]]
PROJCS["WGS 84 / Pseudo-Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY

RasterioIOError: : No such file or directory

In [ ]:
fig, ax = plt.subplots(ncols = 2, nrows=1)
bedrockElevGridIN.plot(ax=ax[0])
surfaceElevGridIN.plot(ax=ax[1])

Add in Control points

In [4]:
#NEED CODE HERE FOR ADDING IN CONTROL Wells MANUALLY
#Add control headerInfo
#Add control description info

# Extract Elevation Data

Extract elevation data from consistent elevation dataset for all wells (lidar or other statewide DEM)

In [5]:
#First, get wells with updated xyz info
    #Check first if xyzData needs to be updated with locations (?)
    #Check which wells in headerData don't have associated lidar data

#statewideLidar =  ow
#mapping.rastertoPoints_extract()

Merge elevation data with headerData table

In [6]:
uniqueWells = headerData['API_NUMBER'].unique()
#xyzData['UniqueWells'] = uniqueWells

headerData = w4h.addElevtoHeader(xyzData, headerData)
##NEED TO UPDATE THIS TO WORK WITH DATA WITH NO XYZ ELEVATION DATA FROM LIDAR
#Change xyz column name to indicate lidar
#Use order of preference: lidar, headerData table?/30/10m DEM?

# Data Cleaning

## First, let's clean up records in the data without the necessary information

Clip data from outside Study Area

In [8]:
headerData = w4h.coords2Geometry(df=headerData, xCol='LONGITUDE', yCol='LATITUDE', crs='EPSG:4269')
#headerData['geometry']=headerData['GEOMETRY'].copy() #old code
headerDataClip = w4h.clipHeader2StudyArea(studyarea=studyAreaIN, headerdata=headerData, headerCRS='EPSG:4269')
headerData = headerDataClip.copy()
headerData

c:\ProgramData\Anaconda3\envs\raster38\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


,API_NUMBER,TOTAL_DEPTH,SECTION,TWP,TDIR,RNG,RDIR,MERIDIAN,QUARTERS,ELEVATION,ELEVREF,COUNTY_CODE,ELEVSOURCE,LATITUDE,LONGITUDE,ELEV_FT,geometry
0,4088244408,48.0,1.0,6.0,N,2.0,W,3.0,SW SW SE,NaN,nan,5.0,nan,39.990276,-89.265556,621.794189,POINT (-89.26556 39.99028)
1,4108370508,62.0,15.0,46.0,N,4.0,E,3.0,NW SW NW,NaN,nan,7.0,nan,40.811356,-89.522789,509.087433,POINT (-89.52279 40.81136)
2,4148227108,30.0,24.0,18.0,N,6.0,E,4.0,NW SE SE,NaN,nan,11.0,nan,41.031700,-89.745049,657.211426,POINT (-89.74505 41.03170)
3,4148237908,175.0,17.0,18.0,N,6.0,E,4.0,SW NW SE,NaN,nan,11.0,nan,40.533516,-89.816887,435.597351,POINT (-89.81689 40.53352)
4,4205883108,501.0,9.0,17.0,N,8.0,W,3.0,SE SW,638.0,GL,17.0,nan,39.932003,-90.007332,635.502869,POINT (-90.00733 39.93200)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71874,1774134712,61.0,2.0,27.0,N,4.0,W,3.0,SW NW SW,487.0,GL,203.0,nan,40.821632,-89.522797,481.388336,POINT (-89.52280 40.82163)
71875,1774134812,50.0,23.0,27.0,N,4.0,W,3.0,NE NE SW,NaN,nan,203.0,nan,40.825806,-89.516159,524.130493,POINT (-89.51616 40.82581)
71876,1774134912,218.0,7.0,28.0,N,2.0,W,3.0,SE NW NW,NaN,nan,203.0,nan,40.905693,-89.388992,643.279053,POINT (-89.38899 40.90569)
71877,1774135012,228.0,12.0,28.0,N,3.0,W,3.0,NE NW NE,NaN,nan,203.0,nan,40.906712,-89.395897,625.861084,POINT (-89.39590 40.90671)


Now, remove data from downholeData table that does not have location information (Since we would not know where to put it anyway)

This should also essentially "clip" the downholeData to the study area, since only study area wells remain in headerData

In [9]:
downholeData = w4h.removeNonlocatedData(downholeData, headerData)

2650467 records removed without location information.
403942 wells remain from 55772 located wells in study area.


Remove headerData rows without surface elevation information (this currently clips data from outside Illinois)

In [10]:
headerData = headerDataClip.copy()
headerData

,API_NUMBER,TOTAL_DEPTH,SECTION,TWP,TDIR,RNG,RDIR,MERIDIAN,QUARTERS,ELEVATION,ELEVREF,COUNTY_CODE,ELEVSOURCE,LATITUDE,LONGITUDE,ELEV_FT,geometry
0,4088244408,48.0,1.0,6.0,N,2.0,W,3.0,SW SW SE,NaN,nan,5.0,nan,39.990276,-89.265556,621.794189,POINT (-89.26556 39.99028)
1,4108370508,62.0,15.0,46.0,N,4.0,E,3.0,NW SW NW,NaN,nan,7.0,nan,40.811356,-89.522789,509.087433,POINT (-89.52279 40.81136)
2,4148227108,30.0,24.0,18.0,N,6.0,E,4.0,NW SE SE,NaN,nan,11.0,nan,41.031700,-89.745049,657.211426,POINT (-89.74505 41.03170)
3,4148237908,175.0,17.0,18.0,N,6.0,E,4.0,SW NW SE,NaN,nan,11.0,nan,40.533516,-89.816887,435.597351,POINT (-89.81689 40.53352)
4,4205883108,501.0,9.0,17.0,N,8.0,W,3.0,SE SW,638.0,GL,17.0,nan,39.932003,-90.007332,635.502869,POINT (-90.00733 39.93200)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71874,1774134712,61.0,2.0,27.0,N,4.0,W,3.0,SW NW SW,487.0,GL,203.0,nan,40.821632,-89.522797,481.388336,POINT (-89.52280 40.82163)
71875,1774134812,50.0,23.0,27.0,N,4.0,W,3.0,NE NE SW,NaN,nan,203.0,nan,40.825806,-89.516159,524.130493,POINT (-89.51616 40.82581)
71876,1774134912,218.0,7.0,28.0,N,2.0,W,3.0,SE NW NW,NaN,nan,203.0,nan,40.905693,-89.388992,643.279053,POINT (-89.38899 40.90569)
71877,1774135012,228.0,12.0,28.0,N,3.0,W,3.0,NE NW NE,NaN,nan,203.0,nan,40.906712,-89.395897,625.861084,POINT (-89.39590 40.90671)


In [11]:
headerData_cleaned = w4h.removenotopo(df=headerData, printouts=True)
headerData = headerData_cleaned.copy()

Well records removed: 72
Number of rows before dropping those without surface elevation information: 71879
Number of rows after dropping those without surface elevation information: 71807


Remove rows from downholeData with no depth information and where depth information is obviously bad (i.e., top depth > bottom depth)

In [12]:
#Drop records with no depth information
donwholeData = w4h.dropnodepth(downholeData, printouts=True)
#Drop records with bad depth information (i.e., top depth > bottom depth) (Also calculates thickness of each record)
donwholeData = w4h.dropbaddepth(downholeData, printouts=True)

Number of rows before dropping those without record depth information: 403942
Number of rows after dropping those without record depth information: 392435
Number of well records without formation information deleted: 11507
Number of rows before dropping those with obviously bad depth information: 403942
Number of rows after dropping those with obviously bad depth information: 392291
Well records deleted: 11651


Drop records with no FORMATION information

In [13]:
downholeData = w4h.dropnoformation(downholeData, printouts=True)

Number of rows before dropping those without FORMATION information: 403942
Number of rows after dropping those without FORMATION information: 403942
Well records deleted: 0


Now we are going to export this data, to have record of cleaned data

In [14]:
downholeData.reset_index(inplace=True,drop=True)
headerData.reset_index(inplace=True,drop=True)

#downholeData.to_csv(str(repoDir)+'/out/downholeData_cleaned'+dateSuffix+'.csv',index_label='ID')
#headerData.to_csv(str(repoDir)+'/out/headerData_cleaned'+dateSuffix+'.csv',index_label='ID')

In [15]:
outData = pd.merge(left = downholeData, right = headerData, on='API_NUMBER')

# Classification

The following flags are used to mark the classification method:
- 0: Not classified
- 1: Specific Search Term Match
- 2: Wildcard match (startTerm) - no context
- 3: Bedrock classification for obvious bedrock
- 4: Wildcard match (startTerm) - with context

In [16]:
#Read in dictionary files for downhole data
specTermsPATH, startTermsPATH = w4h.searchTermFilePaths(dictdir=str(repoDir)+'/resources/', specStartPattern='*SearchTerms-Specific*', startGlobPattern = '*SearchTerms-Start*')

Most Recent version of this file is : SearchTerms-Specific_2022-11-16_essCols.csv
Most Recent version of this file is : SearchTerms-Start.csv


In [17]:
specTerms, startTerms = w4h.readSearchTerms(specfile=specTermsPATH, startfile=startTermsPATH)

Join the dataframes--for the specific search terms, this is the same as classifying them

In [18]:
downholeData_spec = w4h.specificDefine(downholeData, specTerms, printouts=True)
downholeData = downholeData_spec.copy()

Records Classified with full search term: 261362
Records Classified with full search term: 64.7% of data


Create a dataframe with only the records already classified (using the specific search terms in this case, classifiedDF), and one that still needs to be searched (searchDF)

In [19]:
classifedDF, searchDF = w4h.splitDefined(downholeData)
searchDF.shape[0]

142580

Now, do the classification routine on the searchDF database

In [20]:
searchDF = w4h.startDefine(df=searchDF, starterms=startTerms, printouts=True)

Start Term process should be done by 11:42


c:\Users\balikian\LocalData\CodesScripts\Github\wells4hydrogeology\w4h\classify.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLASS_FLAG'].where(~df['FORMATION'].str.startswith(s,na=False),4,inplace=True)
c:\Users\balikian\LocalData\CodesScripts\Github\wells4hydrogeology\w4h\classify.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['INTERPRETATION'].where(~df['FORMATION'].str.startswith(s,na=False),starterms.loc[i,'INTERPRETATION'],inplace=True)


Records classified with start search term: 15630
Records classified with start search term: 10.96% of remaining data


c:\Users\balikian\LocalData\CodesScripts\Github\wells4hydrogeology\w4h\classify.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BEDROCK_FLAG'] = df["INTERPRETATION"] == 'BEDROCK'


Merge specDF and searchDF back together all back in single dataframe

In [21]:
downholeData_Terms = w4h.remergeData(classifieddf=classifedDF, searchdf=searchDF)
downholeData = downholeData_Terms.copy()

Export terms that still need to be defined to csv (along with their counts)

In [22]:
#The outdir should be changed so it doesn't clog up the repository
#classify.export_toBeDefined(df=downholeData, outdir=str(repoDir)+'/out/')

Classify all  data under depth threshold (default is 550') as bedrock (should not be an issue, but just in case)

In [23]:
classifedDF, searchDF = w4h.splitDefined(downholeData)
searchDF = w4h.depthDefine(searchDF, thresh=550, printouts=True)
downholeData_Class = w4h.remergeData(classifieddf=classifedDF, searchdf=searchDF)
downholeData = downholeData_Class.copy()

3.0    4908
Name: CLASS_FLAG, dtype: int64
test
Records classified as bedrock that were deeper than 550': 4908
This represents 3.87% of the unclassified data in this dataframe.


c:\Users\balikian\LocalData\CodesScripts\Github\wells4hydrogeology\w4h\classify.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLASS_FLAG'].mask(df['TOP']>thresh, 3 ,inplace=True) #Add a Classification Flag of 3 (bedrock b/c it's deepter than 550') to all records where the top of the interval is >550'
c:\Users\balikian\LocalData\CodesScripts\Github\wells4hydrogeology\w4h\classify.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BEDROCK_FLAG'].mask(df['TOP']>thresh, True, inplace=True)


Add '0' flag for data still not classified

In [24]:
downholeData = w4h.fillUnclassified(downholeData)

In [25]:
downholeData['CLASS_FLAG'].value_counts()

1.0    261362
0.0    122042
4.0     15630
3.0      4908
Name: CLASS_FLAG, dtype: int64

## Add "Flag" for target interpratations

In [26]:
#dictDir = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\SupportingDocs\\"
targetInterpDF = w4h.readLithologies()

In [27]:
targetInterpDF

,INTERPRETATION,TARGET
0,BEDROCK,DoNotUse
1,FIRECLAY,DoNotUse
2,VOID,DoNotUse
3,CLAY AND STONE,0
4,DIRT AND BEDROCK,DoNotUse
5,DIRT AND STONE,DoNotUse
6,GRAVEL AND STONE,1
7,BOULDER,1
8,BOULDERS AND CLAY,0
9,GRAVEL WITH BOULDERS,1


In [28]:
downholeData = w4h.mergeLithologies(downholedata=downholeData, targinterps=targetInterpDF)

Flags used for target classification purposes:
- -2: No classification 
- -1: Classified, not used/not definitive
- 0: Classified, not target material
- 1: Classified as target material

In [29]:
downholeData['TARGET'].value_counts()

0.0    222363
1.0     54357
Name: TARGET, dtype: int64

Find all unique wells in downhole dataset

In [30]:
#Get Unique well APIs
wellsDF = w4h.getUniqueWells(downholeData)

Number of unique wells in downholeData: 55772


Sort dataset by API Number and Depth of top of record (will be easier to do data analysis with records in the correct order)

In [31]:
downholeData

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,BEDROCK_FLAG,TARGET
0,4088244408,WFORMATIONS,clay brown,9.0,0.0,9.0,CLAY,1.0,False,0.0
1,4088244408,WFORMATIONS,clay gray,36.0,12.0,48.0,CLAY,1.0,False,0.0
2,4088244408,WFORMATIONS,sand brown,3.0,9.0,12.0,UNKNOWN,1.0,False,0.0
3,4088244408,WFORMATIONS,shale gray at,0.0,48.0,48.0,BEDROCK,4.0,True,0.0
4,4108370508,WFORMATIONS,clay,36.0,2.0,38.0,CLAY,1.0,False,0.0
...,...,...,...,...,...,...,...,...,...,...
403937,1774142112,WFORMATIONS,gray clay,53.0,14.0,67.0,CLAY,1.0,False,0.0
403938,1774142112,WFORMATIONS,gray clay,57.0,135.0,192.0,CLAY,1.0,False,0.0
403939,1774142112,WFORMATIONS,gray clay,66.0,68.0,134.0,CLAY,1.0,False,0.0
403940,1774142112,WFORMATIONS,yellow clay,10.0,0.0,10.0,CLAY,1.0,False,0.0


In [32]:
#Make this into a function?
downholeData_sorted = downholeData.sort_values(['API_NUMBER','TOP'])
downholeData_sorted.reset_index(inplace=True, drop=True)
downholeData_sorted = downholeData_sorted[pd.notna(downholeData_sorted["INTERPRETATION"])]
donwholeData.reset_index(inplace=True, drop=True)
donwholeData = downholeData_sorted.copy()

# Get Bedrock Depth and Layer Thickness

Plot just to see them

Reproject and align raster grids for surface elevation and bedrock topo (reproject well data too if needed)

In [ ]:
inGrids = [bedrockElevGridIN, surfaceElevGridIN]
bedrockGrid, surfaceGrid = w4h.alignRasters(unalignedGrids=inGrids, modelgrid=modelGrid)

fig, ax = plt.subplots(ncols = 2, nrows=1)
bedrockGrid.plot(ax=ax[0])
surfaceGrid.plot(ax=ax[1])

Use the surface elevation raster and bedrock elevation raster to get depth to bedrock

In [ ]:
driftThickGrid, layerThickGrid = w4h.get_drift_thick(surface=surfaceGrid, bedrock=bedrockGrid, noLayers=9, plotData=False)

Now, sample each well point (headerData) to get layer thickness, surface elevation, and bedrock 

In [ ]:
headerData = w4h.rastertoPoints_sample(raster=bedrockGrid, ptDF=headerData, newColName='BEDROCK_ELEV_FT')
#headerData['BEDROCK_ELEV_M'] = headerData['BEDROCK_ELEV_FT']* 0.3048

headerData = w4h.rastertoPoints_sample(raster=surfaceGrid, ptDF=headerData, newColName='SURFACE_ELEV_FT')
#headerData['SURFACE_ELEV_M'] = headerData['SURFACE_ELEV_FT']* 0.3048

headerData = w4h.rastertoPoints_sample(raster=driftThickGrid, ptDF=headerData, newColName='BEDROCK_DEPTH_FT')
#headerData['BEDROCK_DEPTH_M'] = headerData['BEDROCK_DEPTH_FT']* 0.3048

headerData = w4h.rastertoPoints_sample(raster=layerThickGrid, ptDF=headerData, newColName='LAYER_THICK_FT')
#headerData['LAYER_THICK_M'] = headerData['LAYER_THICK_FT']* 0.3048

Calculate  all layer depths/elevations at all wells

In [ ]:
headerData = w4h.get_layer_depths(well_metadata=headerData, no_layers=9)

Merge Data from downhole and headerData to enable further calculation

In [ ]:
downholeData = downholeData_sorted.copy()
headerJoinCols = ['API_NUMBER', 'LATITUDE', 'LONGITUDE', 'LONGITUDE_PROJ', 'LATITUDE_PROJ', 'ELEV_FT', 'BEDROCK_ELEV_FT', 'SURFACE_ELEV_FT', 'BEDROCK_DEPTH_FT', 'LAYER_THICK_FT', 'DEPTH_FT_LAYER1', 'DEPTH_FT_LAYER2', 'DEPTH_FT_LAYER3','DEPTH_FT_LAYER4', 'DEPTH_FT_LAYER5', 'DEPTH_FT_LAYER6','DEPTH_FT_LAYER7','DEPTH_FT_LAYER8','DEPTH_FT_LAYER9','geometry']
downholeData_layerInfo = w4h.mergeTables(leftTable=downholeData, rightTable=headerData, rightCols=headerJoinCols, onCol='API_NUMBER', how='inner')

Calculate the top and bottom elevation for each well record in downholeData

In [ ]:
downholeData_layerInfo['BOTTOM'].plot.hist(bins=np.linspace(0,500))

In [ ]:
downholeData = downholeData_layerInfo.copy()

Define function to calculate target thickness in each layer

In [ ]:
res, resdf = w4h.layerTargetThick(downholeData_layerInfo, layer=1)

In [ ]:
resdf

# Work here next!

Now run that function over all the layers, looping through each one

In [ ]:
#THIS CELL WILL NEED TO BE UPDATED
codeTarget='Test'
outDIR = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\MetroEast_HydroGeo\\CodeOutput\\"+codeTarget+"\\"
codeTargShort='Test_'

for i in np.arange(1,10):
    res, res_df = w4h.layer_target_thick(df=downholeData, layer=i)#Run the function defined above for each layer
    outputname = codeTargShort+'Lyr'+str(i)+'.csv' #Create a filename based on the layer and target
    res_df.to_csv(outDIR+outputname)  #Export the file to csv
    #Could also potentially save these to variables for use in following cells

# Interpolate thickness values in each layer

Loop through each layer and interpolate (use same parameters (?))

Ensure rasters align (are co-registered) with grid

# Export

In [ ]:
#Export data 
downhole_bedrockDepth_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\Downhole_BedrockPicks.csv',index_label="ID")
wPermits_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\wPermits_BedrockPicks.csv',index_label="ID")